In [1]:
DEBUG = True
N_DEBUG = 10

OUTPUT_DIR = "./pointing_game_datasets/"

In [2]:
from gpt2outputdataset.detector_radford import DetectorRadford
from detectgpt.detector_detectgpt import DetectorDetectGPT
from detector_guo import DetectorGuo
detector_classes = [DetectorGuo,DetectorRadford]#,DetectorDetectGPT]

from explainer_wrappers import LIME_Explainer, SHAP_Explainer, Anchor_Explainer
explainer_classes = [Anchor_Explainer, LIME_Explainer, SHAP_Explainer ]

In [3]:
import pointing_game_util

In [4]:
import os
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_lg")
nlp.add_pipe('sentencizer')

In [5]:
test = pd.read_pickle("./dataset_test.pkl")
test = test # always load the full dataset! (np.random.shuffle(tokenized_sentences)). slice the actual hybrid_documents if debugging!
documents = test["answer"]
gold_labels = test["author"] == "human_answers" # convention: 0: machine, 1: human, see detector.py

In [6]:
# pd.Series((len(list(nlp(d).sents)) for d in documents)).describe()

In [7]:
# keep a copy of the hybrid documents for the assert in the loop
ref_assert_hybrid_documents, _, _ = pointing_game_util.hybrid(documents.to_list(), gold_labels.to_list(), word_tokenizer=LIME_Explainer(DetectorRadford()).tokenize)


In [8]:
# pd.Series((len(list(nlp(d).sents)) for d in ref_assert_hybrid_documents)).describe()

In [9]:
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [10]:
results = []
for detector_class in detector_classes:
    detector = detector_class()
    print("Initialized " + detector.__class__.__name__)
    for explainer_class in explainer_classes:
        explainer = explainer_class(detector)
        print("Initialized " + explainer.__class__.__name__)

        print("Indexing hybrid documents for " + explainer.__class__.__name__)
        hybrid_documents, tokenized_hybrid_documents, GT = pointing_game_util.hybrid(documents.to_list(), gold_labels.to_list(), word_tokenizer=explainer.tokenize)
        assert (all([a==b for a,b in zip(ref_assert_hybrid_documents,hybrid_documents)])), "(full) Hybrid documents don't match" # tokenized_hybrid_documents differ by design to make the calculation of the pointing game accuracy easier

        if DEBUG:
            hybrid_documents = hybrid_documents[0:N_DEBUG]
            tokenized_hybrid_documents = tokenized_hybrid_documents[0:N_DEBUG]
            GT = GT[0:N_DEBUG]
        
        # write csv (for debug purposes)
        pd.DataFrame(zip(hybrid_documents, tokenized_hybrid_documents, GT), columns=["Hybrid Document", "Tokenized Hybrid Document", "GT"]).to_csv(os.path.join(OUTPUT_DIR, detector.__class__.__name__+ "-"+explainer.__class__.__name__+".csv"),index=False)
        print("Predicting hybrid documents")
        predictions_hybrid = detector.predict_label(hybrid_documents)

        print("Generating explanations on hybrid documents and calculating pointing game accuracy")

        
       # pointing_game_acc = pointing_game_util.get_pointing_game_acc(hybrid_documents, explainer, predictions_hybrid, GT)
        pointing_game_scores = pointing_game_util.get_pointing_game_scores(hybrid_documents, explainer, predictions_hybrid, GT)
      #  print("Pointing game accuracy for {} | {}: {}".format(explainer.__class__.__name__, detector.__class__.__name__, pointing_game_acc))
        results.extend([(explainer.__class__.__name__, detector.__class__.__name__, pointing_game_score) for pointing_game_score in pointing_game_scores])
    
        




Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Initialized DetectorGuo
Initialized Anchor_Explainer
Indexing hybrid documents for Anchor_Explainer
Predicting hybrid documents
Generating explanations on hybrid documents and calculating pointing game accuracy


AttributeError: 'Anchor_Explainer' object has no attribute 'get_fi_scores_batch'

In [ ]:
from scipy.stats.mstats import ttest_1samp

In [ ]:
# some formatting functions
def get_p_asterisks(group):
    val =  group.mean()
   # print(group.name)
    _, p = ttest_1samp(group, popmean=0.5)
    if p <= 0.001:
        return "{:.2f}\\textsuperscript{{***}}".format(val)
    if p <= 0.01:
        return "{:.2f}\\textsuperscript{{**}}".format(val)
    if p <= 0.05:
        return "{:.2f}\\textsuperscript{{*}}".format(val)
    if p > 0.05:
        return "{:.2f}\\textsuperscript{{ns}}".format(val)

def highlight_max(col):
    vals = col.str.extract(r"(-*\d*\.\d*)").astype(float).values.flatten()
    max_val = vals.max()
    return ["font-weight: bold;" if c == max_val else "" for c in vals ]
def df_to_latex(styled_df, caption="TODO", label="TODO"):
    return styled_df.to_latex(environment="table", convert_css=True, clines="all;data", hrules=True, caption=caption, label=label)

In [ ]:
def style_dff(dff, groupby):
    p_results = dff.groupby(groupby).agg(
    {
          "Pointing Game Scores": get_p_asterisks,
        }
    )
    p_results = p_results.style.apply(highlight_max, subset=p_results.columns)
    return p_results


In [ ]:
dff = pd.DataFrame(results, columns=["Explainer", "Detector", "Pointing Game Scores"])

In [ ]:
p_results_aggregate_level = style_dff(dff, groupby=["Explainer"])
display(p_results_aggregate_level)

,Pointing Game Scores
Explainer,
LIME_Explainer,0.50\textsuperscript{ns}
SHAP_Explainer,0.70\textsuperscript{ns}


In [ ]:
p_results = style_dff(dff, groupby=["Explainer", "Detector"])
display(p_results)

In [ ]:
print(df_to_latex(p_results, label="pointing-game-explainer-detector", caption="Scores per detector and explainer"))
print(df_to_latex(p_results_aggregate_level, label="pointing-game-explainer", caption="Scores per explainer"))

\begin{table}
\caption{Scores per detector and explainer}
\label{pointing-game-explainer-detector}
\begin{tabular}{lll}
\toprule
 &  & Pointing Game Scores \\
Explainer & Detector &  \\
\midrule
\multirow[c]{2}{*}{LIME_Explainer} & DetectorGuo & 0.60\textsuperscript{ns} \\
\cline{2-3}
 & DetectorRadford & 0.40\textsuperscript{ns} \\
\cline{1-3} \cline{2-3}
\multirow[c]{2}{*}{SHAP_Explainer} & DetectorGuo & \bfseries 0.80\textsuperscript{ns} \\
\cline{2-3}
 & DetectorRadford & 0.60\textsuperscript{ns} \\
\cline{1-3} \cline{2-3}
\bottomrule
\end{tabular}
\end{table}

\begin{table}
\caption{Scores per explainer}
\label{pointing-game-explainer}
\begin{tabular}{ll}
\toprule
 & Pointing Game Scores \\
Explainer &  \\
\midrule
LIME_Explainer & 0.50\textsuperscript{ns} \\
\cline{1-2}
SHAP_Explainer & \bfseries 0.70\textsuperscript{ns} \\
\cline{1-2}
\bottomrule
\end{tabular}
\end{table}



In [ ]:

# for hybrid_document in hybrid_documents:

#     explainer = LIME_Explainer(detector)
#     explainer.get_explanation_cached(hybrid_document).show_in_notebook()

#     explainer = SHAP_Explainer(detector)
#     shap.text_plot(explainer.get_explanation_cached(hybrid_document))
        